# <span style='color:#FF8C42'>Talk to Database</span>

This notebook provides a user interface to convert natural language into SQL queries. Using this program, you can ask your questions in natural language, receive the equivalent SQL query, and view the results.

## <span style='color:#42c5f5'>Requirements</span>

- Python 3.x
- pyodbc library for connecting to SQL Server
- Access to the OpenRouter API to use language models


## <span style='color:#42c5f5'>Author</span>

<img src="https://media.licdn.com/dms/image/v2/D5603AQEO0rQI1tVtYg/profile-displayphoto-shrink_200_200/profile-displayphoto-shrink_200_200/0/1712605211628?e=1759968000&v=beta&t=KBza-Ntnrj5lQMpTZwBLY1HaeKK6QRsRRHadJmEsra0" 
     alt="Profile Picture" 
     style="width:150px; height:150px; border-radius:50%; object-fit:cover;"/>
     
### <span style='color:#4bc1a5'>Naiem Yousefifard</span>
**AI Engineer | Building Scalable AI-Powered Apps & Automation**

- LinkedIn: https://www.linkedin.com/in/naiem-yousefifard-11086729b
- GitHub: https://github.com/iritman
- YouTube: https://www.youtube.com/@naiemyf7296
- Email: iritman@gmail.com

## <span style='color:#42c5f5'>Import required libraries</span>

First, we import the libraries required to run the program.

In [ ]:
import os
import pyodbc
import json
import requests
from typing import List, Dict, Union

## <span style='color:#42c5f5'>Helper functions</span>

In this section, we define helper functions for clearing the screen and printing the application header.

In [ ]:
def clear_screen():
    """Clear the terminal screen"""
    os.system('cls' if os.name == 'nt' else 'clear')

def print_header():
    """Display the application header"""
    clear_screen()
    print("="*80)
    print("                           TALK TO SQL                           ")
    print("                Natural Language to SQL Query Tool               ")
    print("="*80)
    print("\nConnect to your SQL Server database using natural language prompts.")
    print("Type 'exit' or 'quit' to exit the application.\n")

## <span style='color:#42c5f5'>OpenRouterClient class</span>

This class is used to communicate with the OpenRouter API and leverage language models to convert natural language into SQL queries.

In [ ]:
class OpenRouterClient:
    """Class for communicating with the OpenRouter API"""
    
    def __init__(self, api_key: str, model: str):
        self.api_key = api_key
        self.model = model
        self.base_url = "https://openrouter.ai/api/v1/chat/completions"
    
    def invoke(self, prompt: Union[str, Dict, List[Dict]], **kwargs) -> str:
        # Convert the prompt to messages format
        if isinstance(prompt, str):
            messages = [{"role": "user", "content": prompt}]
        elif isinstance(prompt, dict):
            messages = [prompt]
        else:
            messages = prompt
        
        # Send a request to the OpenRouter API
        response = requests.post(
            url=self.base_url,
            headers={
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json",
            },
            json={
                "model": self.model,
                "messages": messages,
            }
        )
        
        # Check for errors
        if response.status_code != 200:
            raise Exception(f"OpenRouter API error: {response.status_code} - {response.text}")
        
        # Process the response
        response_data = response.json()
        content = response_data.get("choices", [{}])[0].get("message", {}).get("content", "")
        
        return content

## <span style='color:#42c5f5'>Database and OpenRouter API connection settings</span>

In this section, we configure the connection to the SQL Server database and the OpenRouter API.

In [ ]:
# Database connection settings
server_name = "LEGION5"  # Enter your SQL Server name here
database_name = "SampleDB"  # Enter your database name here

# OpenRouter API settings
api_key = "sk-or-v1-xxxxxx"  # Enter your API key here
model_name = "qwen/qwq-32b:free"  # Enter your desired model name here

## <span style='color:#42c5f5'>Connect to SQL Server database</span>

In this section, we connect to the SQL Server database and retrieve the database schema information.

In [ ]:
print_header()
print("\nConnecting to database...")

# Create the connection string
connection_string = f"DRIVER={{SQL Server}};SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes;"

try:
    # Connect to the database
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    
    # Retrieve database schema information
    tables = []
    for table in cursor.tables(tableType='TABLE'):
        if table.table_schem == 'dbo':
            table_name = table.table_name
            tables.append(table_name)
            
    # Retrieve schema details for each table
    schema_info = []
    for table_name in tables:
        columns = []
        for column in cursor.columns(table=table_name):
            column_name = column.column_name
            data_type = column.type_name
            columns.append(f"{column_name} ({data_type})")
        schema_info.append(f"Table: {table_name}\nColumns: {', '.join(columns)}")
        
    schema_text = "\n\n".join(schema_info)
    print("✅ Successfully connected to database.")
    
    # Display database schema information
    print("\nDatabase Schema Information:")
    print("-" * 80)
    print(schema_text)
    print("-" * 80)
except Exception as e:
    print(f"\n❌ Failed to connect to the database: {str(e)}")

## <span style='color:#42c5f5'>Connect to the OpenRouter API</span>

In this section, we connect to the OpenRouter API so we can use language models to convert natural language into SQL queries.

In [ ]:
print("\nConnecting to OpenRouter API...")
try:
    # Create the OpenRouter client
    llm = OpenRouterClient(api_key=api_key, model=model_name)
    print("✅ Successfully connected to OpenRouter API.")
except Exception as e:
    print(f"\n❌ Failed to connect to OpenRouter API: {str(e)}")

## <span style='color:#42c5f5'>Define the system prompt for generating SQL queries</span>

In this section, we define the system prompt for generating SQL queries. The prompt includes the database schema information and the necessary instructions to generate SQL queries.

In [ ]:
# Create the system prompt for generating SQL queries
sql_system_prompt = """You are an expert SQL query generator in the SQL Server (T-SQL) database. 
Your task is to convert natural language questions into SQL queries. 

Here is the database schema information:
{schema}

EXTREMELY IMPORTANT: You MUST return ONLY a valid JSON object with the following structure and nothing else:
{{
    "query": "YOUR SQL QUERY HERE"
}}

The query should be valid for SQL Server (T-SQL).
IMPORTANT: SQL Server does not support LIMIT keyword. Use TOP instead.

DO NOT include any explanations, markdown formatting, or additional text outside the JSON structure.
DO NOT use code blocks (```json) around your response.
DO NOT include any thinking, reasoning, or explanations in your response.
DO NOT include any text before or after the JSON object.
ONLY return the JSON object with the query field.
ENSURE your response is a complete, valid JSON object that can be parsed directly.
"""

## <span style='color:#42c5f5'>Function to process the user's question and generate a SQL query</span>

This function receives the user's question and, using the OpenRouter API, generates the equivalent SQL query.

In [ ]:
def process_user_question(question: str, llm: OpenRouterClient, schema_text: str, sql_system_prompt: str) -> str:
    """Process the user's question and generate a SQL query"""
    print("\nProcessing your request...")

    try:
        # Generate the SQL query using the OpenRouter API
        sql_messages = [
            {"role": "system", "content": sql_system_prompt.format(schema=schema_text)},
            {"role": "user", "content": question}
        ]
        sql_response = llm.invoke(sql_messages)
        sql_response = sql_response.strip()

        # Try to find the last JSON block in the response text
        last_open_brace = sql_response.rfind("{")
        last_close_brace = sql_response.rfind("}")
        sql_query = sql_response  # fallback to full response text

        if last_open_brace != -1 and last_close_brace > last_open_brace:
            json_str = sql_response[last_open_brace:last_close_brace+1]
            try:
                response_json = json.loads(json_str)
                sql_query = response_json.get("query", sql_query).strip()
            except json.JSONDecodeError:
                print("⚠️ Warning: Could not parse JSON. Using raw response as fallback.")

        # Remove extra whitespace
        sql_query = sql_query.strip()
        return sql_query

    except Exception as e:
        print(f"\n❌ An error occurred while processing your request: {str(e)}")
        return None

## <span style='color:#42c5f5'>Function to execute the SQL query and display results</span>

This function executes the SQL query and displays the results as JSON.

In [ ]:
import pandas as pd

def execute_query_and_show_results(sql_query: str, cursor: pyodbc.Cursor) -> None:
    """Execute the SQL query and display results"""
    if not sql_query:
        return

    # Print the generated SQL query
    print("\n✅ Generated SQL Query:")
    print("-" * 80)
    print(sql_query)
    print("-" * 80)

    try:
        # Execute the query
        cursor.execute(sql_query)

        # Extract column names (replace empty ones with "Data")
        columns = [col[0] if col[0] else "Data" for col in cursor.description]

        # Fetch all rows
        rows = cursor.fetchall()

        # Convert to Pandas DataFrame
        df = pd.DataFrame.from_records(rows, columns=columns)

        print("\n📊 Query Results:")
        print("-" * 80)
        print(df.to_string(index=False))
        print("-" * 80)

    except Exception as e:
        # Print error if query execution fails
        print(f"\n❌ Error executing query: {str(e)}")

## <span style='color:#42c5f5'>Main program loop</span>

In this section, we run the main program loop, which receives the user's questions, generates the equivalent SQL query, and displays the results.

In [ ]:
# Main program loop
try:
    while True:
        print("\n" + "-"*80)
        print("\n🔍 Enter your question or command:")
        print("   - Type 'exit' or 'quit' to exit")
        question = input("\n> ")
        
        if question.lower() in ["exit", "quit"]:
            break
            
        if not question.strip():
            continue
        
        # Process the user's question and generate the SQL query
        sql_query = process_user_question(question, llm, schema_text, sql_system_prompt)
        
        # Execute the SQL query and display the results
        execute_query_and_show_results(sql_query, cursor)
        
except KeyboardInterrupt:
    print("\n\nProgram interrupted by user. Exiting...")
except Exception as e:
    print(f"\n❌ An unexpected error occurred: {str(e)}")
finally:
    print("\nThank you for using TalkToSQL!")

## <span style='color:#42c5f5'>Usage examples</span>

In this section, we present a few example questions and their equivalent SQL queries.

### <span style='color:#42d5b5'>Example 1: Retrieve all members</span>

Question: Show the list of all members

SQL query:
```sql
SELECT * FROM Members
```

### <span style='color:#42d5b5'>Example 2: Retrieve members with a specific condition</span>

Question: Show members older than 30 years

SQL query:
```sql
SELECT * FROM Members WHERE Age > 30
```

### <span style='color:#42d5b5'>Example 3: Retrieving Information with Table Joins</span>

Question: Show a list of all courses each member has enrolled in

SQL query:
```sql
SELECT m.FirstName, m.LastName, c.CourseName
FROM Members m
JOIN Enrollments e ON m.MemberID = e.MemberID
JOIN Courses c ON e.CourseID = c.CourseID
ORDER BY m.LastName, m.FirstName
```